# Overview

Welcome to a demo of snapshot and real time replication to Databricks.

Use this notebook customized schema, data, workload, and **legacy** Arcion.

**NOTE**: **Databricks Personal Access Token** and **Arcion License** are required. 

- Initial Setup
  - Open `Table of Contents` (Outline)
  - Enter `Arcion License`
  - Enter `Databricks Personal Access Token`
  - Click `Run All`
  - Click `View` -> `Results Only`
  - Click `View` -> `Web Terminal`, 
    - enter `tmux attach`.  
      - If fails with `session not found`, then wait a bit retry.
    - In the `tmux`'s console window, `htop` will be displayed during the setup.
    - Once the setup is complete, Arcion snapshot summary will be displayed.
    - Wait for the setup to finish and the snapshot to complete. 
    - Takes about 5 minutes in for the setup to finish.
- Iterate with the following:
  - Configure Schema and Data
  - Configure Workload
  - Configure Arcion

## Where is Data in Databricks
  - Spark (Delta Lake) uses **Hive Meta Store** catalog: 
    - Open new tab Catalog -> hive_metastore -> <your username>
    - find ycsbdense and ycsbsparse tables 
  - Lakehouse uses **Unity Catalog** catalog: 
    - Open new tab Catalog -> <your username> 
    - find ycsbdense and ycsbsparse tables 

## Frequent Demo Configurations
- Step 1
  - Click Real-Time
  - Run just Arcion
  - Change YCSB Size
  - Watch real-time performance
- Step 2
  - Click Unity Catalog target
  - Select full replication mode
  - Run just Arcion

# Personal Compute Cluster

Choose at least 16GB of RAM for a demo.

Processes use RAM.  The following is the minimum RAM usage.  The server needs enough RAM to avoid swapping.
- Databricks: 5GB 
- SQL Server: 2GB
- Arcion: 10% of server RAM.

Note:
- `vmstat 5`.  any non zero metrics under the `si` and `so` columns (swap in and swap out) indicate RAM shortage. 
- DBR 13 does not print output of subprocess.run

In [ ]:
# prep python env
import subprocess
import math
import pandas as pd
import re
import ipywidgets as widgets
import os
import pathlib
import json
import requests
from ipywidgets import HBox, VBox, Label

# setup GUI elements

repl_mode = widgets.Dropdown(options=['snapshot', 'real-time', 'full'],value='snapshot',
    description='Replication:',
)
cdc_mode = widgets.Dropdown(options=['change', 'cdc'],value='change',
    description='CDC Method:',
)
ram_percent = widgets.BoundedIntText(value=10,min=10,max=80,
    description='RAM %:',
)

snapshot_threads = widgets.BoundedIntText(value=1,min=1,max=8,
    description='Snapshot Threads:',
)

realtime_threads = widgets.BoundedIntText(value=1,min=1,max=8,
    description='Real Time Threads:',
)    

delta_threads = widgets.BoundedIntText(value=1,min=1,max=8,
    description='Delta Snapshot Threads:',
)    

dbx_destinations = widgets.Dropdown(options=['null', 'deltalake', 'unitycatalog'],value='null',
    description='Destinations:',
)
dbx_staging = widgets.Dropdown(options=['dbfs'],value='dbfs',
    description='Staging:',
)

sparse_cnt = widgets.BoundedIntText(value=4,min=1,max=1000,
    description='Tbl End:',
)
sparse_cntstart = widgets.BoundedIntText(value=1,min=1,max=1000,
    description='Tbl Start:',
)

sparse_fieldcount = widgets.BoundedIntText(value=50,min=0,max=9000,
    description='# of Fields:',
)
sparse_fieldlength = widgets.BoundedIntText(value=10,min=1,max=1000,
    description='Field Len:',
)

sparse_tps = widgets.BoundedIntText(value=1,min=0,max=1000,
    description='TPS:',
)
sparse_threads = widgets.BoundedIntText(value=1,min=1,max=8,
    description='Threads:',
)
sparse_recordcount = widgets.Text(value="2K",
    description='Rec Cnt:',
)

sparse_fillpct = widgets.IntRangeSlider(value=[0,0],min=0,max=100,step=1,
    description='Fill Range:', orientation='horizontal', readout=False
)

dense_cnt = widgets.BoundedIntText(value=2,min=1,max=1000,
    description='Tbl End:',
)
dense_cntstart = widgets.BoundedIntText(value=1,min=1,max=1000,
    description='Tbl Start:',
)

dense_fieldcount = widgets.BoundedIntText(value=10,min=0,max=9000,
    description='# of Fields:',
)
dense_fieldlength = widgets.BoundedIntText(value=100,min=1,max=1000,
    description='Field Len:',
)
dense_recordcount = widgets.Text(value="1K",
    description='Rec Cnt:',
)

dense_tps = widgets.BoundedIntText(value=1,min=0,max=1000,
    description='TPS:',
)
dense_threads = widgets.BoundedIntText(value=1,min=1,max=8,
    description='Threads:',
)

dense_fillpct = widgets.IntRangeSlider(value=[0,100],min=0,max=100,step=1,
    description='Fill Range:', orientation='horizontal', readout=False
)

dbx_spark_url = widgets.Textarea(value='',
    description='Spark URL:',
)

dbx_databricks_url = widgets.Textarea(value='',
    description='Databricks URL:',
)

dbx_hostname = widgets.Textarea(value='',
    description='Hostname:',
)

src_username = widgets.Textarea(value='',
    description='SRC User:',
)

dbx_username = widgets.Textarea(value='',
    description='DST User:',
)

arcion_license = widgets.Textarea(value='',
    description='Lic',
)

arcion_download_url = widgets.Textarea(value='https://arcion-releases.s3.us-west-1.amazonaws.com/general/replicant/replicant-cli-24.01.25.7.zip',
    description='Download URL',
)

dbx_access_token = widgets.Password(value='',
    description='Access Token',
)

dbx_default_catalog = widgets.Textarea(value='',
    description='HMS Catalog',
)


# cluster where the notebook is running to auto populate the destinations
spark_url=""
databricks_url=""
workspaceUrl=""
username=""
try:
    cluster_id = spark.conf.get("spark.databricks.clusterUsageTags.clusterId")
    workspace_id =spark.conf.get("spark.databricks.clusterUsageTags.clusterOwnerOrgId")

    # clusterName = spark.conf.get("spark.databricks.clusterUsageTags.clusterName")

    workspaceUrl = json.loads(dbutils.notebook.entry_point.getDbutils().notebook().getContext().toJson())['tags']['browserHostName']

    # below does not work on GCP
    # sc.getConf().getAll() to see what is avail
    # workspaceUrl = spark.conf.get("spark.databricks.workspaceUrl") # host name

    http_path = f"sql/protocolv1/o/{workspace_id}/{cluster_id}"

    spark_url=f"jdbc:spark://{workspaceUrl}:443/default;transportMode=http;ssl=1;httpPath={http_path};AuthMech=3;UID=token;"
    databricks_url=f"jdbc:databricks://{workspaceUrl}:443/default;transportMode=http;ssl=1;httpPath={http_path};AuthMech=3;UID=token;"

except:
    pass
dbx_spark_url.value = spark_url
dbx_databricks_url.value = databricks_url
dbx_hostname.value = workspaceUrl

try:
    username = spark.sql("SELECT current_user()").collect()[0][0]
    dbx_username.value = re.sub('[.@]','_',username)
    src_username.value = re.sub('[.@]','_',username)
except:
    src_username.value='arcsrc'
    dbx_username.value='arcdst'

try:
    dbx_default_catalog.value=spark.conf.get("spark.databricks.sql.initial.catalog.name")
except:
    pass

# check arcion license via os env
try:
    arclicenv=os.environ["ARCION_LICENSE"]
    if arclicenv != "": 
        arcion_license.value=arclicenv
except:
    pass

# check arcion license via dbx widget
try:
    arclicwidget=dbutils.widgets.get("Arcion License")
    if arclicwidget != "": 
        arcion_license.value=arclicwidget
        arcion_license.disabled = True
except:
    pass

# check access token via dbx widget
try:
    acctokwidget=dbutils.widgets.get("Access Token")
    if acctokwidget != "": 
        dbx_access_token.value=acctokwidget
        dbx_access_token.disabled = True
except:
    pass

# check if os env has ARCION_LICENSE
try:
    arclicenv=os.getenv('ARCION_LICENSE')
    if arclicenv != "": 
        arcion_license.value=arclicenv
except:
    pass

# gcp does not change cwd to notebook path
pwd_result= subprocess.run(f"""pwd""",capture_output = True, text = True )
if (pwd_result.stdout == "/databricks/driver\n"):
    notebookpath="/Workspace" + str(pathlib.Path(dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()).parent)
else:
    notebookpath = None

# optional MLflow
experiment_id=None
try:
    import mlflow
    experiment_id=dbutils.widgets.get("Experiment ID")
except:
    pass

# Setup
  - Enter `Arcion License`
  - Enter `Personal Access Token` (generate **One Day** and delete afterwards)
  - Click **Menu Bar** ->  Run -> Run All Below

## Configure

In [ ]:
# enter license and DBX personal access token
VBox([HBox([Label('Arcion'), arcion_license, arcion_download_url]),
      HBox([Label('DBX'), dbx_access_token, dbx_default_catalog]),
      HBox([Label('Username'), src_username, dbx_username]),
      HBox([Label('Workspace'), dbx_spark_url, dbx_databricks_url, dbx_hostname, ]),
       ])

## Start

In [ ]:
# setup tmux, arcion, ycsb, sql server
subprocess.run(f""". ./bin/setup-tmux.sh; setup_tmux '{dbx_username.value}'""",shell=True,executable="/usr/bin/bash",cwd=notebookpath)
subprocess.run(f"""bin/install-sqlserver.sh""",shell=True,executable="/usr/bin/bash",cwd=notebookpath)
subprocess.run(f"""bin/download-jars.sh""",shell=True,executable="/usr/bin/bash",cwd=notebookpath)
subprocess.run(f"""ARCION_LICENSE='{arcion_license.value}' ARCION_DOWNLOAD_URL='{arcion_download_url.value}' bin/install-arcion.sh""",shell=True,executable="/usr/bin/bash",cwd=notebookpath)
subprocess.run(f"""bin/install-ycsb.sh""",shell=True,executable="/usr/bin/bash",cwd=notebookpath)
subprocess.run(f"""export SRCDB_ARC_USER={src_username.value}; . ./demo/sqlserver/run-ycsb-sqlserver-source.sh; ping_sql_cli;""",shell=True,executable="/usr/bin/bash",cwd=notebookpath)
subprocess.run(f"""export SRCDB_ARC_USER={src_username.value}; . ./demo/sqlserver/run-ycsb-sqlserver-source.sh; create_user;""",shell=True,executable="/usr/bin/bash",cwd=notebookpath)
subprocess.run(f"""export SRCDB_ARC_USER={src_username.value}; . ./demo/sqlserver/run-ycsb-sqlserver-source.sh; set_sqlserver_ram '{dbx_username.value}';""",shell=True,executable="/usr/bin/bash",cwd=notebookpath)
subprocess.run(f"""export SRCDB_ARC_USER={src_username.value}; bin/install-prometheus.sh""",shell=True,executable="/usr/bin/bash",cwd=notebookpath)

# Schema and Data

Existing tables will be appended with additional rows if the `Fill Range` is the same.  
Increase the `Table Count` to create additional tables.  

The following options are available:
- Table count (Table Cnt): The number of tables to create.  
  - Table names are `ycsbdense`, `ycsbdense2`, `ycsbdense3`, ... and `ycssparse`, `ycsbdense2`, and `ycsbdense3` ...
- Number of Fields (# of Fields): The number of fields per table.  
  - The field names are `FIELD0`, `FIELD1`, `FIELD2`, ...
  - Note the use of `K`,`M`,`B` ... suffix at the end.
- Field Length (Field Len): The length of random character data populated per field.  
  - Note the use of `K`,`M`,`B` ... suffix at the end.
- Record Count (Rec Cnt): The number of records per table generated.
  - Note the use of `K`,`M`,`B` ... suffix at the end.
- Fill Range: The relative start and end range of fields that are populated with data.  Be default: 
    - sparse tables are all NULLs by having the fill range be 0% to 0% ranges
    - dense tables have all fields populated by having the fill range be 0% to 100% of ranges 

```sql
[localhost][arcsrc] 1> \describe ycsbsparse
+-------------+-------------+-----------+-------------+----------------+-------------+
| TABLE_SCHEM | COLUMN_NAME | TYPE_NAME | COLUMN_SIZE | DECIMAL_DIGITS | IS_NULLABLE |
+-------------+-------------+-----------+-------------+----------------+-------------+
| dbo         | YCSB_KEY    | int       |          10 |              0 | NO          |
| dbo         | FIELD0      | text      |  2147483647 |         [NULL] | YES         |
| dbo         | FIELD1      | text      |  2147483647 |         [NULL] | YES         |
```

## Configure
Make changes below and click `Run All Below`.  

In [ ]:
# show YCSB Data Controls
VBox([HBox([Label('Sparse'), sparse_cntstart,sparse_cnt, sparse_fieldcount, sparse_fieldlength, sparse_recordcount, sparse_fillpct]),
    HBox([Label('Dense'),  dense_cntstart, dense_cnt, dense_fieldcount, dense_fieldlength, dense_recordcount, dense_fillpct])])

## Start

In [ ]:
# run load_sparse_data_cnt and load_dense_data_cnt 
subprocess.run(f"""export SRCDB_ARC_USER={src_username.value}; 
    . ./demo/sqlserver/run-ycsb-sqlserver-source.sh; 
    y_fieldcount={sparse_fieldcount.value} 
    y_fieldlength={sparse_fieldlength.value}  
    y_recordcount={sparse_recordcount.value} 
    y_fillstart={math.ceil((sparse_fillpct.value[0] * sparse_fieldcount.value) / 100)}      
    y_fillend={int((sparse_fillpct.value[1] * sparse_fieldcount.value) / 100)}      
    load_sparse_data_cnt {sparse_cnt.value} {sparse_cntstart.value};
    y_fieldcount={dense_fieldcount.value} 
    y_fieldlength={dense_fieldlength.value} 
    y_recordcount={dense_recordcount.value} 
    y_fillstart={math.ceil((dense_fillpct.value[0] * dense_fieldcount.value) / 100)}      
    y_fillend={int((dense_fillpct.value[1] * dense_fieldcount.value) / 100)}      
    load_dense_data_cnt {dense_cnt.value} {dense_cntstart.value};
    dump_schema;
    list_table_counts""",
    shell=True,executable="/usr/bin/bash",cwd=notebookpath) 
# show tables
pd.read_csv (f"/var/tmp/{src_username.value}/sqlserver/config/list_table_counts.csv",header=None, names= ['table name','min key','max key','field count'])

# Workload

Choose the options in the UI and run the cell below it to start the workload (YCSB).  

YCSB update (workload A) controls for Dense and Sparse table groups separated. Each group has a separate control.  However, all of the tables in the group use the same controls.  
1. Each table's TPS (throughput per second)
   1. 0=fast as possible
   2. 1=1 TPS
   3. 10=10 TPS
2. Each table's threads (concurrency) used to achieve the desired TPS.

## Configure

In [ ]:
# show YCSB run controls
VBox([HBox([Label('Sparse'), sparse_tps, sparse_threads]), HBox([Label('Dense'),  dense_tps, dense_threads])])

## Start

In [ ]:
# start/restart YCSB run
subprocess.run(f"""export SRCDB_ARC_USER={src_username.value};
    . ./demo/sqlserver/run-ycsb-sqlserver-source.sh; 
    kill_ycsb;
    y_target_sparse={sparse_tps.value} 
    y_target_dense={dense_tps.value} 
    y_threads_sparse={sparse_threads.value} 
    y_threads_dense={dense_threads.value} 
    y_fieldlength_sparse={sparse_fieldlength.value} 
    y_fieldlength_dense={dense_fieldlength.value} 
    start_ycsb;""",
    shell=True,executable="/usr/bin/bash",cwd=notebookpath)

# Arcion

Choose the options in the UI and run the cell below it to start the replication.  

The following control are avail in the demo.  
- Arcion - replication type and CDC methods  
- Threads - control the parallelism.
- Target - null, unity catalog or delta lake

NOTE: Full mode does not work at this time.

For SQL Server, change tracking, cdc are available for demo.  

Performance is mainly controlled by the thread count by the extract and apply process.
Additional controls are customizable via modifying the YAML files directly below.
- [CDC YAML files](./demo/sqlserver/yaml/cdc/)
- [Change Tracking YAML files](./demo/sqlserver/yaml/change/)

## Configure
Make changes below and click `Run All Below`.  

In [ ]:
# show Arcion and DBX controls
VBox([
      HBox([Label('RAM'), ram_percent]),
      HBox([Label('Modes'), repl_mode, cdc_mode]),
      HBox([Label('Target'), dbx_destinations, dbx_staging ]),
      HBox([Label('Threads'), snapshot_threads, realtime_threads, delta_threads]),
      ])

## Start

In [ ]:
# start/restart Arcion
if ( f"{dbx_access_token.value}" == "" ) and ( f"{dbx_destinations.value}" != "null" ):
    print("personal access token not entered.")
else:
    # start a new run
    print (f"""{cdc_mode.value} {repl_mode.value}""")
    arcion_run = subprocess.run(f"""export ARCION_DOWNLOAD_URL='{arcion_download_url.value}';
    export SRCDB_ARC_USER={src_username.value};
    . ./demo/sqlserver/run-ycsb-sqlserver-source.sh; 
    kill_arcion;
    echo prog_dir=$PROG_DIR arcion_bin=$ARCION_BIN;
    cd $PROG_DIR;
    a_repltype='{repl_mode.value}'
    REPLICANT_MEMORY_PERCENTAGE='{ram_percent.value}.0'
    SRCDB_SNAPSHOT_THREADS='{snapshot_threads.value}' 
    SRCDB_REALTIME_THREADS='{realtime_threads.value}' 
    SRCDB_DELTA='{delta_threads.value}'
    DSTDB_TYPE='{dbx_destinations.value}'
    DSTDB_STAGE='{dbx_staging.value}'
    DBX_SPARK_URL='{dbx_spark_url.value}'
    DBX_DATABRICKS_URL='{dbx_databricks_url.value}'
    DBX_ACCESS_TOKEN='{dbx_access_token.value}'
    DBX_HOSTNAME='{dbx_hostname.value}'
    DBX_DBFS_ROOT='/{dbx_username.value}'
    DBX_USERNAME='{dbx_username.value}'
    start_{cdc_mode.value}_arcion;""",
    shell=True,executable="/usr/bin/bash",cwd=notebookpath)

# MLFLow

Save the artifacts in MLFlow.

Artifacts are collected for 5 min (600 sec).

## Start

In [ ]:
import mlflow
import time
import os
import numpy as np
from multiprocessing import Process

def log_prom_metric(prom_metric_url=None,metric_prefix="",metric_step=None):
    # there is a limit on the number of metrics that you can log in a single log_batch call. This limit is typically 1000. 
    # timestamp=If unspecified, the number of milliseconds since the Unix epoch is used.
    # step=If unspecified, the default value of zero is used
    contents = requests.get(prom_metric_url)
    all_metrics = {}
    metrics_count = 0
    for line in contents.text.splitlines():
        if line.startswith("#"):
            continue
        key_val=line.rsplit(' ',1)  # split from the end in case the key has spaces
        all_metrics[re.sub('[" {}=,]',"_",key_val[0])]=float(key_val[1])
        metrics_count += 1
    return(all_metrics)


def start_mlflow(max_intervals=5,experiment_id=None, log_interval_sec=60):
    # stop previous run
    mlflow_run = mlflow.active_run()
    if not(mlflow_run is None):
        print(f"""stopping previous MLflow {mlflow_run.info.run_id}""")
        mlflow.end_run()

    # start a new run
    if experiment_id == '':
        experiment_id=None
    mlflow.start_run(experiment_id=experiment_id, log_system_metrics=True)

    # arcion
    mlflow.log_param("arcion_download_url", arcion_download_url.value)
    mlflow.log_param("srcdb_arc_user", src_username.value)
    mlflow.log_param("repl_type", repl_mode.value)
    mlflow.log_param("replicant_memory_percentage",ram_percent.value)
    mlflow.log_param("srcdb_snapshot_threads",snapshot_threads.value)
    mlflow.log_param("srcdb_realtime_threads",realtime_threads.value) 
    mlflow.log_param("srcdb_delta",delta_threads.value)
    mlflow.log_param("dstdb_type",dbx_destinations.value)
    mlflow.log_param("dstdb_stage",dbx_staging.value)
    mlflow.log_param("dbx_spark_url",dbx_spark_url.value)
    mlflow.log_param("dbx_databricks_url",dbx_databricks_url.value)
    mlflow.log_param("dbx_hostname",dbx_hostname.value)
    mlflow.log_param("dbx_dbfs_root",dbx_username.value)
    mlflow.log_param("dbx_username",dbx_username.value)

    # workload
    mlflow.log_param("sparse_tps",sparse_tps.value)
    mlflow.log_param("dense_tps",dense_tps.value)
    mlflow.log_param("sparse_threads",sparse_threads.value)
    mlflow.log_param("dense_threads",dense_threads.value)

    # cluster
    try:
        mlflow.log_param("spark.databricks.clusterUsageTags.clusterNodeType",spark.conf.get("spark.databricks.clusterUsageTags.clusterNodeType"))
        mlflow.log_param("spark.databricks.clusterUsageTags.cloudProvider",  spark.conf.get("spark.databricks.clusterUsageTags.cloudProvider"))
    except:
        pass

    # schema
    dataset_source=f"/var/tmp/{src_username.value}/sqlserver/config/list_table_counts.csv"
    mlflow.log_artifact(dataset_source)
    
    # data
    dataset_shape = pd.read_csv(dataset_source, header=None, names= ['table name','min key','max key','field count'])
    dataset = mlflow.data.from_pandas(dataset_shape, source=dataset_source)
    mlflow.log_input(dataset, context="training")    

    # wait to end
    # TODO: Make this smarter by checking whether the process is still running
    wait_count=0
    while wait_count < max_intervals:
        mlflow.log_metrics(metrics=log_prom_metric(prom_metric_url="http://localhost:9399/metrics"))
        mlflow.log_metrics(metrics=log_prom_metric(prom_metric_url="http://localhost:9100/metrics"))
        time.sleep(log_interval_sec)
        wait_count += 1

    # upload the rest of the artifacts generated /var/tmp/{src_username.value}/sqlserver/logs
    # experiment done
    mlflow.end_run()

p = Process(target=start_mlflow, kwargs={"experiment_id":experiment_id})
p.start()   

# Manually Kill Processes
Uncomment below to kill desired processes

In [ ]:
# subprocess.run(f""". ./demo/sqlserver/run-ycsb-sqlserver-source.sh; kill_arcion';""",shell=True,executable="/usr/bin/bash",cwd=notebookpath)
# subprocess.run(f""". ./demo/sqlserver/run-ycsb-sqlserver-source.sh; kill_ycsb';""",shell=True,executable="/usr/bin/bash",cwd=notebookpath)